objectives
- find the what contigs were classified as viral by genomad
- determine the lengths
- find alignments in NCBI blast

In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "ProgressMeter",
    "uCSV",
    "StatsPlots",
    "StatsBase",
    "FASTX"
]
# Pkg.activate(; temp=true)
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")

# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

sample_directories = joinpath.(data_dir, "SRA", joint_metadata[!, "Run"])

In [ ]:
joint_genomad_results = DataFrames.DataFrame()
# sample_directory = first(sample_directories)
ProgressMeter.@showprogress for sample_directory in sample_directories
    genomad_virus_summary = joinpath(sample_directory, "genomad", "final.contigs.fastg.gfa_summary", "final.contigs.fastg.gfa_virus_summary.tsv")
    genomad_results = DataFrames.DataFrame(uCSV.read(genomad_virus_summary, delim='\t', header=1, typedetectrows=100)...)
    genomad_results[!, "sample_id"] .= basename(sample_directory)
    append!(joint_genomad_results, genomad_results, promote=true)
end
joint_genomad_results[!, "seq_name"] = string.(joint_genomad_results[!, "seq_name"])

In [ ]:
StatsBase.describe(joint_genomad_results[!, "length"])

In [ ]:
p = StatsPlots.scatter(
    joint_genomad_results[!, "length"],
    joint_genomad_results[!, "virus_score"],
    legend=false,
    title = "Genomad Viral Classification Confidence by Assembled Contig Length",
    titlefontsize=10,
    ylabel = "Classification Confidence",
    xlabel = "Assembled Contig Length",
    dpi=300
)
StatsPlots.savefig(p, joinpath(base_dir, "data", "results", "genomad-classification-confidence-by-contig-length.png"))
display(p)